In [3]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

In [13]:
# !pip install pathway bokeh panel --quiet  # Uncomment if needed

import pandas as pd
import pathway as pw
import datetime
import bokeh.plotting
import panel as pn
from bokeh.models import ColumnDataSource

# Step 1: Prepare data
df = pd.read_csv("/content/dataset.csv")

df = df.rename(columns={
    "TrafficConditionNearby": "TrafficLevel",
    "ID": "LotID"
})

df["Timestamp"] = pd.to_datetime(
    df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
    format="%d-%m-%Y %H:%M:%S"
)

cols_needed = [
    "Timestamp", "Occupancy", "Capacity", "QueueLength",
    "VehicleType", "TrafficLevel", "IsSpecialDay",
    "Latitude", "Longitude", "LotID", "SystemCodeNumber"
]

df[cols_needed].to_csv("parking_stream.csv", index=False)

# Step 2: Schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    VehicleType: str
    TrafficLevel: str
    IsSpecialDay: int
    Latitude: float
    Longitude: float
    LotID: int
    SystemCodeNumber: str

# Step 3: Load CSV stream
data = pw.demo.replay_csv(
    "parking_stream.csv",
    schema=ParkingSchema,
    input_rate=1000
)

# Step 4: Time parsing
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# Step 5: Tumbling window with pricing logic
windowed = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_max = pw.reducers.max(pw.this.Occupancy),
        occ_min = pw.reducers.min(pw.this.Occupancy),
        cap = pw.reducers.max(pw.this.Capacity)
    )
    .with_columns(
        price = 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)[["t", "price"]]

# Step 6: Bokeh Visualization
pn.extension()
source = ColumnDataSource(data=dict(t=[], price=[]))

# ✅ Must accept all keyword arguments
def update_plot(*, key, row, time, is_addition):
    if is_addition:
        source.stream({"t": [row["t"]], "price": [row["price"]]}, rollover=1000)

# ✅ Use `on_change=...` (keyword argument)
pw.io.subscribe(windowed, on_change=update_plot)

plot = bokeh.plotting.figure(
    height=400,
    width=800,
    title="Daily Dynamic Parking Price",
    x_axis_type="datetime",
    x_axis_label="Time",
    y_axis_label="Price ($)"
)
plot.line("t", "price", source=source, line_width=2, color="navy", legend_label="Price")
plot.scatter("t", "price", source=source, size=6, color="red")
plot.legend.location = "top_left"

pn.Column(plot).servable()

# Step 7: Run streaming pipeline
pw.run()


Output()

TypeError: update_plot() got an unexpected keyword argument 'key'